In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer ##for Later
from sklearn.model_selection import train_test_split #splits dataset into training and test data 
# Load the dataset
file_path = "/Users/behruzazimov/Downloads/resume_data.csv"

# Read dataset
data = pd.read_csv(file_path)

# Show basic info
print("Dataset loaded successfully!")
data.info()
data.head()



In [33]:
#removing unnecessary rows
data.drop(columns=['address', 'passing_years', 'result_types', 'company_urls', 'start_dates', 'end_dates', 'extra_curricular_organization_links',
'online_links', 'issue_dates', 'expiry_dates', 'age_requirement', 'matched_score'], inplace = True)

In [36]:
#double check that the columns were dropped
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9544 entries, 0 to 9543
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   career_objective                     4740 non-null   object
 1   skills                               9488 non-null   object
 2   educational_institution_name         9460 non-null   object
 3   degree_names                         9460 non-null   object
 4   educational_results                  9460 non-null   object
 5   major_field_of_studies               9460 non-null   object
 6   professional_company_names           9460 non-null   object
 7   related_skils_in_job                 9460 non-null   object
 8   positions                            9460 non-null   object
 9   locations                            9460 non-null   object
 10  responsibilities                     9544 non-null   object
 11  extra_curricular_activity_types      3426 n

,career_objective,skills,educational_institution_name,degree_names,educational_results,major_field_of_studies,professional_company_names,related_skils_in_job,positions,locations,...,role_positions,languages,proficiency_levels,certification_providers,certification_skills,﻿job_position_name,educationaL_requirements,experiencere_requirement,responsibilities.1,skills_required
0,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['N/A'],['Electronics'],['Coca-COla'],[['Big Data']],['Big Data Analyst'],['N/A'],...,NaN,NaN,NaN,NaN,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,Technical Support\nTroubleshooting\nCollaborat...,NaN
1,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['N/A', 'N/A']","['Mathematics', 'Statistics']",['BIB Consultancy'],"[['Data Analysis', 'Business Analysis', 'Machi...",['Business Analyst'],['N/A'],...,NaN,NaN,NaN,NaN,NaN,Machine Learning (ML) Engineer,M.Sc in Computer Science & Engineering or in a...,At least 5 year(s),Machine Learning Leadership\nCross-Functional ...,NaN
2,NaN,"['Software Development', 'Machine Learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['N/A'],['Electronics/Telecommunication'],['Axis Bank Limited'],"[['Unified Payment Interface', 'Risk Predictio...",['Software Developer (Machine Learning Enginee...,['N/A'],...,NaN,NaN,NaN,NaN,NaN,"Executive/ Senior Executive- Trade Marketing, ...",Master of Business Administration (MBA),At least 3 years,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...
3,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...","['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,[None],['Computer Applications'],"['Company Name ï¼ City , State', 'Company Name...","[['accounts receivables', 'banking', 'G/L Acco...","['Accountant', 'Accounts Receivable Clerk', 'M...","['City, State', 'City, State', 'City, State', ...",...,NaN,NaN,NaN,NaN,NaN,Business Development Executive,Bachelor/Honors,1 to 3 years,Apparel Sourcing\nQuality Garment Sourcing\nRe...,Fast typing skill\nIELTSInternet browsing & on...
4,Professional accountant with an outstanding wo...,"['Analytical reasoning', 'Compliance testing k...",['Kent State University'],['Bachelor of Business Administration'],['3.84'],['Accounting'],"['Company Name', 'Company Name', 'Company Name...","[['collections', 'accounts receivable', 'finan...","['Staff Accountant', 'Senior Accountant', 'Tax...","['City, State', 'City, State', 'City, State', ...",...,"[None, None, None, None]",NaN,NaN,['Ohio Notary Public'],[None],Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,At least 4 years,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...


In [ ]:
def searchResume(resumes,keywords):
    keyword_list = [kw.strip().lower() for kw in keywords.split(",")]
    

In [ ]:
#all the code, no need to run
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer ##for Later
import re
# Load the dataset
file_path = "/Users/behruzazimov/Downloads/resume_data.csv"

# Read dataset
data = pd.read_csv(file_path)

# Show basic info
print("Dataset loaded successfully!")
data.info()
data.head()

data.drop(columns=['address', 'passing_years', 'result_types', 'company_urls', 'start_dates', 'end_dates', 'extra_curricular_organization_links',
'online_links', 'issue_dates', 'expiry_dates', 'age_requirement', 'matched_score'], inplace = True)

##Handle Missing Values
print("\nMissing Values Before Cleaning:")
print(data.isnull().sum())

##remove
